In [15]:
from aocd.models import Puzzle
from pprint import pprint
import random
from copy import deepcopy

In [16]:
puzzle_input = Puzzle(2021, 12).input_data.split("\n")
small_test_input = [
	"start-A",
	"start-b",
	"A-c",
	"A-b",
	"b-d",
	"A-end",
	"b-end"
]
medium_test_input = [
	"dc-end",
	"HN-start",
	"start-kj",
	"dc-start",
	"dc-HN",
	"LN-dc",
	"HN-end",
	"kj-sa",
	"kj-HN",
	"kj-dc",
]
large_test_input = [
	"fs-end",
	"he-DX",
	"fs-he",
	"start-DX",
	"pj-DX",
	"end-zg",
	"zg-sl",
	"zg-pj",
	"pj-he",
	"RW-he",
	"fs-DX",
	"pj-RW",
	"zg-RW",
	"start-pj",
	"he-WI",
	"zg-he",
	"pj-fs",
	"start-RW",
]

In [17]:
def generate_connection_directory(connection_list):
	connection_map = {}
	for connection in connection_list:
		connection_start = connection.split("-")[0]
		connection_end = connection.split("-")[1]
		
		if connection_start not in connection_map:
			connection_map[connection_start] = [connection_end]
		if connection_start in connection_map:
			if connection_end not in connection_map[connection_start]:
				connection_map[connection_start].append(connection_end)

		if connection_end not in connection_map:
			connection_map[connection_end] = [connection_start]
		if connection_end in connection_map:
			if connection_start not in connection_map[connection_end]:
				connection_map[connection_end].append(connection_start)

	return connection_map

In [35]:
def generate_path(input_directory):
	# since small caves can only be visited at most once, and big caves any number of times
	# disregard any small caves which only have one connection to another small cave,
	# as these ones cant be visited in the first place.
	new_input_directory = deepcopy(input_directory)
	new_input_directory = {x:new_input_directory[x]
							for x,new_input_directory[x] in new_input_directory.items()
							if len(new_input_directory[x]) > 1 or
							new_input_directory[x][0] == new_input_directory[x][0].upper()}

	path = "start,"
	current_cave = "start"
	restart = False

	while "end" not in path:
		possible_next_caves = [x for x in new_input_directory[current_cave] if x in new_input_directory and (x == x.upper() or x not in path)]
		if not possible_next_caves:
			# all possible caves are small caves which have already been explored. You have reached a cul-de-sac. Revert step
			previous_cave = path.split(",")[-2]
			new_input_directory[previous_cave] = [val for val in new_input_directory[previous_cave] if val != current_cave]
			path = ",".join(path.split(",")[:-2]) + ","
		else:
			next_cave = random.choice(possible_next_caves)
			path += f"{next_cave},"
			current_cave = next_cave
		# if restart:
		# 	path = "start,"
		# 	current_cave = "start"
		# 	restart = False
		# 	continue


	return path[:-1]  # remove comma behind "end"

In [19]:
# The difference between this function and generate_path() is that this function allows for a *single* small
# cave (provided it is not the start and end cave) to be visitied twice.

def generate_path_2(input_directory):
	new_input_directory = deepcopy(input_directory)
	# new_input_directory = {x:new_input_directory[x]
	# 						for x,new_input_directory[x] in new_input_directory.items()
	# 						if len(new_input_directory[x]) > 1 or
	# 						new_input_directory[x][0] == new_input_directory[x][0].upper()}
	
	# pprint(new_input_directory)

	path = "start,"
	current_cave = "start"
	restart = False
	small_cave_visited_twice = False

	while "end" not in path:
		possible_next_caves = [x for x in new_input_directory[current_cave] 
								if (x == x.upper() or (x != "start" and (x not in path or not small_cave_visited_twice)))
								or x == "end"]
		# print(current_cave, possible_next_caves)
		if not possible_next_caves:
			# all possible caves are small caves which have already been explored. You have reached a cul-de-sac. Restart path finding process.
			# print("Cul de sac. Restarting...")
			restart = True
		else:
			next_cave = random.choice(possible_next_caves)
			if next_cave == next_cave.lower() and next_cave in path:
				small_cave_visited_twice = True
			path += f"{next_cave},"
			current_cave = next_cave

		if restart:
			path = "start,"
			current_cave = "start"
			restart = False
			small_cave_visited_twice = False
			continue


	return path[:-1]  # remove comma behind "end"

In [40]:
def calculate_num_paths(input_directory, part):
	paths = []
	already_in_paths = 0
	threshold = 10000
	# if after <threshold> iterations no new path has been found, it is highly likely that all possible paths have been explored.
	while already_in_paths < threshold:
		path = generate_path(input_directory) if part == "a" else generate_path_2(input_directory)
		if path in paths:
			already_in_paths += 1
		else:
			paths.append(path)
			already_in_paths = 0

	print(len(paths))
	# pprint(set(paths))

In [41]:
small_test_connection_directory = generate_connection_directory(small_test_input)
medium_test_connection_directory = generate_connection_directory(medium_test_input)
large_test_connection_directory = generate_connection_directory(large_test_input)
puzzle_input_connection_directory = generate_connection_directory(puzzle_input)

In [43]:
# part a
calculate_num_paths(puzzle_input_connection_directory, part="a")

5884


In [44]:
# part b
